<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_merlin_02-deploying-multi-stage-recsys-with-merlin-systems/nvidia_logo.png" style="width: 90px; float: right;">

# Deploying Online Multi-Stage RecSys with Triton Inference Server

At this point, we expect that you have already executed the first notebook `01-Building-Online-Multi-Stage-Recsys-Components.ipynb` and exported all the required files and models. 

We are going to generate recommended items for a given user query (user_id) by following the steps described in the figure below.

![img](./img/OnlineMultiStageRecsys.png)

We will serve the multi-stage recommender on [Triton Inference Server](https://github.com/triton-inference-server/server)(TIS) easily and efficiently.

### Import required libraries and functions

In [1]:
import os
import numpy as np
import feast
import shutil

from nvtabular import ColumnSchema, Schema
from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.session_filter import FilterCandidates
from merlin.systems.dag.ops.softmax_sampling import SoftmaxSampling
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ops.unroll_features import UnrollFeatures
from merlin.systems.triton.utils import send_triton_request

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-19 22:38:34.948298: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/local/lib/python3.8/dist-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/f

In [2]:
# Define output path for data
BASE_DIR = os.environ['PWD']
FEATURE_STORE_DIR = os.path.join(BASE_DIR, "feature_repo/")
TRITON_MODEL_REPO = os.path.join(BASE_DIR, "models/")

DATA_DIR = os.path.join(BASE_DIR, "data")
DLRM_DIR = os.path.join(DATA_DIR, "dlrm")
QUERY_TOWER_DIR = os.path.join(DATA_DIR, "query_tower")
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_RETRIEVAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, "retrieval")

## Define Triton Ensemble
In order to run our Recsys in Triton, we need to assemble the pieces that will run together as an ensemble.

### Setup Triton Model Repo

Define paths for ranking and retrieval model in the Triton Model Repo. We need to move/copy our trained models from the `DATA_DIR` to the `TRITON_MODEL_REPO` so it can be consumed by Triton on startup.

In [6]:
retrieval_model_path = os.path.join(TRITON_MODEL_REPO, "1-user-embeddings/1/model.savedmodel/")
ranking_model_path = os.path.join(TRITON_MODEL_REPO, "5-ranking/1/model.savedmodel/")

# Copy over pretrined Query Tower Model to our Triton Model Repository
if not os.path.isdir(retrieval_model_path):
    shutil.copytree(QUERY_TOWER_DIR, retrieval_model_path)

# Copy over pretrined DLRMfor ranking to our Triton Model Repository
if not os.path.isdir(ranking_model_path):
    shutil.copytree(DLRM_DIR, ranking_model_path)

### Explore Triton Model Repo
Below we will take a look at the multi-stage ensemble that functions as a DAG of operations within Triton.

In [7]:
import seedir as sd

sd.seedir(
    TRITON_MODEL_REPO,
    style='lines',
    itemlimit=10,
    depthlimit=5,
    exclude_folders=['.ipynb_checkpoints', '__pycache__'],
    sort=True
)

models/
├─0-query-user-features/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─1-user-embeddings/
│ ├─1/
│ │ └─model.savedmodel/
│ │   ├─assets/
│ │   ├─keras_metadata.pb
│ │   ├─saved_model.pb
│ │   └─variables/
│ │     ├─variables.data-00000-of-00001
│ │     └─variables.index
│ └─config.pbtxt
├─2-redis-vss-candidates/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─3-query-item-features/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─4-unroll-features/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─5-ranking/
│ ├─1/
│ │ └─model.savedmodel/
│ │   ├─.merlin/
│ │   │ └─input_schema.json
│ │   ├─assets/
│ │   ├─keras_metadata.pb
│ │   ├─saved_model.pb
│ │   └─variables/
│ │     ├─variables.data-00000-of-00001
│ │     └─variables.index
│ └─config.pbtxt
├─6-softmax-sampling/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
└─ensemble-model/
  ├─1/
  └─config.pbtxt


The subfolders (**starting with 0-6**) in the model repo above represent distinct stages in the RecSys ensemble.

- `0-query-user-features/` - fetch user features from Redis.
- `1-user-embeddings/` - generate user embeddings from the Query Tower (Tensorflow) model.
- `2-redis-vss-candidates/` - perform VSS to find KNN items using RediSearch.
- `3-query-item-features/` - fetch item features from Redis.
- `4-unroll-features/` - combine and unroll user and item features.
- `5-ranking/` - rank the top User/Item pairs with the DLRM (Tensorflow) model.
- `6-softmax-sampling/` - sort all inputs in descending order, introduce some randomization via softmax sampling, and return top-k ordered items.
- `ensemble-model/`

The `ensemble-model` contains the orchestration of all of the individual steps. To learn more about general Triton model repo structure [check this out](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_repository.md).

## Starting Triton Server

Now we can deploy all the models as an ensemble model to Triton Inference Serve [TIS](https://github.com/triton-inference-server). After we export the ensemble, we are ready to start the TIS. You can start triton server by using the following command on your terminal:

```bash
tritonserver --model-repository=/workdir/models --backend-config=tensorflow,version=2
```

*For the `--model-repository` argument, specify the path to the Triton Model Repo stored in the var `TRITON_MODEL_REPO`.* This command will launch the server and load all the models to the server. Once all the models are loaded successfully, you should see `READY` status printed out in the terminal for each loaded model.

### Retrieving Recommendations from Triton

Once our models are successfully loaded to the TIS, we can now easily send a request to TIS and get a response for our query with our simple Python `client`.

Let's send a request to TIS for a given `user_id_raw` value. If you make multiple requests in a row for same user, you should see slightly different results based on the randomness introduced via softmax sampling!

In [14]:
!python client.py --user 23

Finding recommendations for User 23
Recommended Product Ids in 0.036287784576416016 seconds
[[507]
 [375]
 [108]
 [339]
 [395]
 [281]
 [669]
 [397]
 [ 79]
 [570]
 [249]
 [173]
 [124]
 [236]
 [314]
 [ 48]]


## Conclusion

That's it! You finished deploying an online multi-stage Recommender Systems on Triton Inference Server with Redis!